### User Define Model Class
- perents class: nn.Module
- 필수 오버라이딩
    - __init__(): 모델 층 구성, 전개
    - forward(): 순방향 학습 진행 코드
 - 동적 모델
    - container 모듈 둥 nn.ModuleList() 사용해서 동적으로 Layer 추가
        - 연결이 안되어있다??
        - Sequential은 연결되어있지만 저 친구는 아닌ㅁ
        - forward 기능 미제공
        - layer 어쩌구 저쩌구 나중에 기록하자...

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optima

from torchinfo import summary
from torchmetrics.regression import R2Score, MeanSquaredError
from torchmetrics.classification import F1Score, Accuracy, ConfusionMatrix

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
torch.manual_seed(14)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

- basic create ANN model
    - input layer: input node is len(feature)
    - output layer: output node is len(label)
    - hidden layer: fixed

- model structure
    - datasets: feature dynamic, label dynamic, regression
    - input layer   : input dynamic,   output dynamic,  activation function: ReLU, sigmoid (To solve gradient vanishing problem)
    - hidden layer  : dynamic, activation function: ReLu, sigmoid
    - output layer  : input dynamic, output dynamic,   activation function: None (마지막에는 확률값을 기반으로 결과를 도출하기에, 분류는 필요함. sigmoid, softmax)
    - input, output, hidden 모든것이 동적, hidden perceptron도 동적

- - -
- 모델 이름: DynamicModel
- 부모 class: nn.Module
- parameter: in_in, out_out, h_in=[], h_outs=[]
- - -

In [30]:
# hidden layer가 동적인 모델
class DynamicMyModel(nn.Module):
    # callback function
    def __init__(self, in_in, in_out, out_out, h_in=[], h_out=[]):
        super().__init__()
        
        self.input_layer = nn.Linear(in_in, h_in[0] if len(h_in) else in_out)
        self.h_layers = nn.ModuleList()
        for idx in range(len(h_in)):
            self.h_layers.append(nn.Linear(h_in[idx], h_out[idx]))
        # nn.ModuleList: 층을 동적으로 넣을때
        self.output_layer = nn.Linear(h_out[-1] if len(h_out) else in_out, out_out)
        
    # forward learning
    # callback funtion
    # 전달인자: 학습용 데이터셋
    def forward(self, x):
        y = self.input_layer(x)
        y = F.relu(y)
        if len(self.h_layers):
            for layer in self.h_layers:
                y = layer(F.relu(y))
        y = self.output_layer(y)
        return y

In [31]:
h_in , h_out = [30, 50, 70], [50, 70, 30]
model = DynamicMyModel(3, 5, 2, h_in, h_out)


In [35]:
print(model, end='\n\n')

for name, param in model.named_parameters():
    print(f"[{name}]\n{param}\n")

DynamicMyModel(
  (input_layer): Linear(in_features=3, out_features=30, bias=True)
  (h_layers): ModuleList(
    (0): Linear(in_features=30, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=70, bias=True)
    (2): Linear(in_features=70, out_features=30, bias=True)
  )
  (output_layer): Linear(in_features=30, out_features=2, bias=True)
)

[input_layer.weight]
Parameter containing:
tensor([[-1.8903e-02,  1.4697e-01,  3.3553e-01],
        [-4.2390e-01,  4.6174e-01, -4.1965e-01],
        [ 5.1141e-03,  4.2055e-01, -2.7052e-01],
        [-5.2810e-01,  5.6565e-01,  5.7650e-01],
        [ 5.7447e-04,  1.7841e-01, -4.0783e-01],
        [ 1.7680e-01,  4.8439e-01,  4.7958e-01],
        [-4.4693e-01, -2.3760e-01, -5.7395e-01],
        [ 2.5227e-01,  2.0006e-01, -3.5936e-01],
        [ 2.4061e-01, -2.4257e-01, -4.2605e-01],
        [ 4.2655e-01, -3.9283e-01, -4.3722e-01],
        [-2.3995e-01,  2.7364e-01, -1.5960e-01],
        [-4.9221e-01,  2.6890e-01,  1.1480e-01],
     

In [33]:
data_ts = torch.FloatTensor([[1, 3, 5], [2, 4, 6], [3, 5, 7], [4, 6, 8]])
target_ts = torch.FloatTensor([[7, 9], [8, 10], [9, 11], [10, 18]])

In [34]:
y_pred = model(data_ts)
print(y_pred)

tensor([[ 0.1247, -0.1213],
        [ 0.1495, -0.0897],
        [ 0.1737, -0.0582],
        [ 0.1985, -0.0258]], grad_fn=<AddmmBackward0>)
